# Dense Retrieval on HotpotQA

## Installation

In [ ]:
!pip install sentence-transformers faiss-cpu accelerate datasets tqdm evaluate 

## Imports

In [ ]:
from sentence_transformers import SentenceTransformer 
